<a href="https://colab.research.google.com/github/Coldestadam/Adam-AIPND-CNN/blob/master/Holy_Grail_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn

import matplotlib.pyplot as plt

# Uploading Dataset

In [2]:
from google.colab import files
files.upload()

Saving concrete_data.csv to concrete_data.csv


{'concrete_data.csv': b'Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength\r\n540.0 ,0.0 ,0.0 ,162.0 ,2.5 ,1040.0 ,676.0 ,28 ,79.99 \r\n540.0 ,0.0 ,0.0 ,162.0 ,2.5 ,1055.0 ,676.0 ,28 ,61.89 \r\n332.5 ,142.5 ,0.0 ,228.0 ,0.0 ,932.0 ,594.0 ,270 ,40.27 \r\n332.5 ,142.5 ,0.0 ,228.0 ,0.0 ,932.0 ,594.0 ,365 ,41.05 \r\n198.6 ,132.4 ,0.0 ,192.0 ,0.0 ,978.4 ,825.5 ,360 ,44.30 \r\n266.0 ,114.0 ,0.0 ,228.0 ,0.0 ,932.0 ,670.0 ,90 ,47.03 \r\n380.0 ,95.0 ,0.0 ,228.0 ,0.0 ,932.0 ,594.0 ,365 ,43.70 \r\n380.0 ,95.0 ,0.0 ,228.0 ,0.0 ,932.0 ,594.0 ,28 ,36.45 \r\n266.0 ,114.0 ,0.0 ,228.0 ,0.0 ,932.0 ,670.0 ,28 ,45.85 \r\n475.0 ,0.0 ,0.0 ,228.0 ,0.0 ,932.0 ,594.0 ,28 ,39.29 \r\n198.6 ,132.4 ,0.0 ,192.0 ,0.0 ,978.4 ,825.5 ,90 ,38.07 \r\n198.6 ,132.4 ,0.0 ,192.0 ,0.0 ,978.4 ,825.5 ,28 ,28.02 \r\n427.5 ,47.5 ,0.0 ,228.0 ,0.0 ,932.0 ,594.0 ,270 ,43.01 \r\n190.0 ,190.0 ,0.0 ,228.0 ,0.0 ,932.0 ,670.0 ,90 ,42.33 \r\n304.0 ,76.0 ,0.0 ,228.0 ,0.0 ,932.0 ,670.0 ,28 

# Looking at Dataset

In [24]:
df = pd.read_csv("/content/concrete_data.csv")
df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [25]:
df.shape

(1030, 9)

#Standardizing Dataset

In [27]:
label_cols = ['Age', 'Strength']
columns_to_standardize = df.drop(columns=label_cols).columns
means = df.loc[:,columns_to_standardize].mean()
stds = df.loc[:,columns_to_standardize].std()
df.loc[:,columns_to_standardize] = (df.loc[:,columns_to_standardize]-means)/stds
df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,28,79.99
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,28,61.89
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,270,40.27
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,365,41.05
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,360,44.30


#Separating our features and targets

Age and Strength are targets

In [29]:
features = df.drop(columns=['Age', 'Strength'])
targets = df.loc[:, ["Age", "Strength"]]

In [30]:
features.mean().values

array([ 2.43222379e-15, -8.51368598e-16,  3.83781464e-16,  1.84674282e-15,
       -9.64115519e-16,  6.81870957e-15,  1.23257122e-14])

# Creating model using PyTorch

When implementing the active learning algorithm, it will use PyTorch to retrain the model with newly sampled data. This will be a simple neural network, with no hidden layers. This really constructs two Linear Regression models.

In [40]:
class neural_network(nn.Module):
    def __init__(self, num_features, num_labels):
        super(neural_network, self).__init__()
        self.fc = nn.Linear(num_features, num_labels)

    def forward(self, x):
        # Takes input x, and passes that through the network
        return self.fc(x)

# THis gets the number of columns in the features df
num_features = features.shape[1]

# This gets the number of targets
num_targets = targets.shape[1]

model = neural_network(num_features, num_targets)

print(model)

neural_network(
  (fc): Linear(in_features=7, out_features=2, bias=True)
)


In [41]:
import torch.optim as optim

# Defining Loss function
loss_fn = nn.MSELoss()

# Defining Optimizer with l2 regularization since the dataset is quite small
optimizer = optim.Adam(model.parameters(), weight_decay=0.001, lr=0.001)

In [42]:
# Function to train and optimize model
def train_model(model, num_epochs, loss_fn, optimizer, training_features, training_labels):
    for epoch in range(1, num_epochs+1):
        # Setting optimizer gradients to zero
        optimizer.zero_grad()

        # Getting predictions
        preds = model(training_features)

        # Calculating loss with L2 regularization
        loss = loss_fn(preds, training_labels)

        # Calculating Gradients
        loss.backward()

        # Taking a gradient descent step
        optimizer.step()

        #print('Epoch:{}\t MSE_Loss:{}'.format(epoch, loss.item()))

# Active Learning Sampling Technique

I will be implementing all the alogorithms from this paper: https://arxiv.org/pdf/1808.04245.pdf

Especially *Improved Greedy Sampling* (iGS) technique from section 2.3 of the paper.

## GSx Algorithm

In [65]:
from sklearn.metrics import pairwise_distances_argmin_min

# This is the function that computes distance for GSx algorithm (Equation 2 in Section 2.1)
def distance_GSx(unlabeled_sample, K_list):
    """
    This function is meant to select the current unlabelled sample and calculate it's distance to all the other 
    samples in K, where the samples in K are all the samples that will be used to label, then construct a model
    to label the other unlabelled data.

    In this function we will select minimum distance from the unlabelled samples to all selected samples in K.
    This minimum distance will be returned for all non-selected unlabelled distance, which then it will select
    the unlabelled sample with the MAXIMUM of the minimum distances, and that unlabelled sample will be selected to
    be in K. So K will contain the samples that are incremently farthest away from eachother, therefore K will contain
    a good variance that will represent the dataset. Then all K values will be labelled by hand.

    params:
        unlabeled_sample (np.array) - The unlabelled sample that is used to find the closest sampled K value
        K_list (list) - A list of all current chosen samples that will be labelled.

    returns:
        The minimum distance of all the distances between the unlabelled sample and all K samples 
    """
    distances = []
    for k_sample in K_list:

        # This takes the euchilidean distance from the labeled sample(k) and unlabeled sample(m)
        distance = np.linalg.norm(k_sample - unlabeled_sample)
        distances.append(distance)
    return min(distances)

# This will use GSx method from paper to get the first batch of data to then label
def GSx_Sampling(features, K, true_dataset=None, label_cols=None):
    """
    params:
        features (pd.DataFrame) - This is the dataset that is unlabelled, which will provide samples to hand label
                                before constructing a model to then use predictions to label
        K (int) - This is the total amount of samples that would be hand labelled before constructing a model
        true_dataset (pd.DataFrame) - This is the real dataset to sample labels from since this is for simulation
        label_cols (list) - A list of all label column names in true_dataset to label the K samples

    returns:
        This will return a labelled dataset of the samples that were selected and unlabelled dataset if the real dataset is given.
        If not, it will provide the features that were selected.
    """

    # Dictionary that is going to hold the chosen samples and their indices
    K_set = {'samples':[], 'indices':[]}

    # Finding the centroid by taking the mean of all variables
    centroid = features.mean().values

    # using sklearn, finding closest sample to centroid
    closest_idx, _ = pairwise_distances_argmin_min(centroid.reshape(1, -1), features)
    
    # Grabbing the index integer in the array 
    closest_idx = closest_idx[0]

    # Placing first k sample and index into K_set
    first_k = features.iloc[closest_idx].values
    K_set['samples'].append(first_k)
    K_set['indices'].append(closest_idx)
    
    
    # Removing first_k from unlabeled features
    features = features.drop(closest_idx, axis=0)

    # This will run K-1 times to select K-1 samples for K
    for k in range(1, K):
        # This will save all the distances for this iteration
        GSx_distances = []
        
        for sample_idx in features.index:
            
            # Computing the minimum distances of current unlabelled sample and all K samples
            min_distance = distance_GSx(features.loc[sample_idx].values, K_set['samples'])
                
            # Appending the minimum distance
            GSx_distances.append(min_distance)

        # Grabbing the sample idx with the largest minimum distance to a single k value
        new_k_sample_idx = features.index[np.argmax(GSx_distances)]
        
        # Placing new k sample and index into K_set
        new_k_sample = features.loc[new_k_sample_idx].values
        K_set['samples'].append(new_k_sample)
        K_set['indices'].append(new_k_sample_idx)

        # Delecting the new k sample from features
        features = features.drop(new_k_sample_idx, axis=0)
    
    # Creating a dataframe of K samples
    K_df = pd.DataFrame(K_set['samples'], columns=features.columns, index=K_set['indices'])
    
    # This will return K samples and their labels if they are given
    if type(true_dataset) == pd.core.frame.DataFrame and type(label_cols) == list:
        labels = true_dataset.loc[K_set['indices'], label_cols]
        return K_df, labels

    # This will only return the K samples if the true dataset was not given
    return K_df

def GSx_Algorithm(features, K, model, true_dataset=None, label_cols=None, epochs=None):
    """
    params:
        features (pd.DataFrame) - This is the dataset that is unlabelled, which will provide samples to hand label
                                before constructing a model to then use predictions to label
        K (int) - This is the total amount of samples that would be hand labelled before constructing a model
        model (PyTorch Model) - This is the model that will be trained with the selected K samples
        true_dataset (pd.DataFrame) - This is the real dataset to sample labels from since this is for simulation
        label_cols (list) - A list of all label column names in true_dataset to label the K samples

    returns:
        This will return a labelled dataset of the samples that were selected and unlabelled dataset if the real dataset is given.
        If not, it will provide the features that were selected.
    """

    # Dictionary that is going to hold the chosen samples and their indices
    K_set = {'samples':[], 'indices':[]}

    # Finding the centroid by taking the mean of all variables
    centroid = features.mean().values

    # using sklearn, finding closest sample to centroid
    closest_idx, _ = pairwise_distances_argmin_min(centroid.reshape(1, -1), features)
    
    # Grabbing the index integer in the array 
    closest_idx = closest_idx[0]

    # Placing first k sample and index into K_set
    first_k = features.iloc[closest_idx].values
    K_set['samples'].append(first_k)
    K_set['indices'].append(closest_idx)
    
    
    # Removing first_k from unlabeled features
    features = features.drop(closest_idx, axis=0)

    # This will run K-1 times to select K-1 samples for K
    for k in range(1, K):
        print('Getting k={} sample'.format(k))
        # This will save all the distances for this iteration
        GSx_distances = []
        
        for sample_idx in features.index:
            
            # Computing the minimum distances of current unlabelled sample and all K samples
            min_distance = distance_GSx(features.loc[sample_idx].values, K_set['samples'])
                
            # Appending the minimum distance
            GSx_distances.append(min_distance)

        # Grabbing the sample idx with the largest minimum distance to a single k value
        new_k_sample_idx = features.index[np.argmax(GSx_distances)]
        
        # Placing new k sample and index into K_set
        new_k_sample = features.loc[new_k_sample_idx].values
        K_set['samples'].append(new_k_sample)
        K_set['indices'].append(new_k_sample_idx)

        # Delecting the new k sample from features
        features = features.drop(new_k_sample_idx, axis=0)
    
    # Creating a dataframe of K samples
    K_features = pd.DataFrame(K_set['samples'], columns=features.columns, index=K_set['indices'])
    
    # Getting K_labels
    K_labels = true_dataset.loc[K_set['indices'], label_cols]

    # Converting K features and labels to PyTorch tensors
    K_torch_features = torch.tensor(K_features.values, dtype=torch.float)
    K_torch_labels = torch.tensor(K_labels.values, dtype=torch.float)

    print('Training Model with all K values')
    
    train_model(model, epochs, loss_fn, optimizer, K_torch_features, K_torch_labels)
    K_data = pd.concat([K_features, K_labels], axis=1)
    return model, K_data, features

In [70]:
GSx_K, GSx_data, unlabeled_data = GSx_Algorithm(features, K=100, model=model, true_dataset=df, label_cols=['Age', 'Strength'], epochs=50)

Getting k=1 sample
Getting k=2 sample
Getting k=3 sample
Getting k=4 sample
Getting k=5 sample
Getting k=6 sample
Getting k=7 sample
Getting k=8 sample
Getting k=9 sample
Getting k=10 sample
Getting k=11 sample
Getting k=12 sample
Getting k=13 sample
Getting k=14 sample
Getting k=15 sample
Getting k=16 sample
Getting k=17 sample
Getting k=18 sample
Getting k=19 sample
Training Model with all K values


## GSy Algorithm

In [43]:
# This is the function that computes distance for GSy algorithm (Equation 2 in Section 2.2)
def distance_GSy(unlabeled_prediction, K_0_labels):
    """
    This function is meant to select the current unlabelled sample models prediction, f(x), and calculate the distance
    of each label of the K samples. The model f(x), is trained on the samples and labels of K_0 sampled using GSx.

    In this function we will select minimum distance from predictions from the unlabelled samples to all lables in K.
    This minimum distance will be returned for all non-selected unlabelled samples, which then it will select
    the unlabelled sample with the MAXIMUM of the minimum distances, and that unlabelled sample will be selected to
    be in K. So K will contain the samples that have higher variation in the target space.

    params:
        unlabeled_sample (np.array) - The unlabelled prediction that is used to find the closest sampled K label
        K_0_labels (pd.Series or pd.DataFrame) - A Series or DataFrame of all labels collected for K_0 labels from GSx algo

    returns:
        The minimum distance of all the distances between the unlabelled sample's prediction and all K samples' predictions
    """
    distances = []

    # If there are multiple targets, will use euchlidean distance
    if K_0_labels.values.ndim > 1:
        for k_label in K_0_labels.values:

            # This takes the euchilidean distance from the labeled sample(k) and unlabeled sample(m)
            distance = np.linalg.norm(unlabeled_prediction - k_label)
            distances.append(distance)
    # If the targets are 1-dimensional then you will just take the absolute value difference
    else:
        for k_label in K_0_labels.values:
            distances.append(np.abs(unlabeled_prediction - k_label))

    return min(distances)

# This will use GSy method from paper to get a model able to make predictions from 
def GSy_Algorithm(features, K, model, true_dataset=None, label_cols=None, K_0=None, epochs=None):
    """
    params:
        features (pd.DataFrame) - This is the dataset that is unlabelled, which will provide samples to hand label
                                before constructing a model to then use predictions to label
        K (int) - This is the total amount of samples that would be hand labelled before constructing a model
        model (PyTorch Model) - This is the model that will be trained overtime
        true_dataset (pd.DataFrame) - This is the real dataset to sample labels from since this is for simulation
        label_cols (list) - A list of all label column names in true_dataset to label the K samples
        K_0 (int) - The number of initial k samples that are sampled from the GSx algorithm to then train the model.
                    According to the authors, it is set to the number of features in the dataset.
        epochs (int) - The number of epochs to train the model for each update

    returns:
        This will return a PyTorch model trained on the K labelled Samples, the leftover unlabelled data to predict,
        and the Final K dataset
    """
    
    # Initializing K_0 to the number of features
    if K_0 == None:
        K_0 = features.shape[1]

    print('Running GSx Algorithm to get K_0 data')
    # Running GSx algorithm to collect K_0 samples
    K_0_features, K_0_labels = GSx_Sampling(features, K=K_0, true_dataset=true_dataset, label_cols=label_cols)

    # Removing labeled K_0 data from features
    features = features.drop(index = K_0_features.index)

    # Creating Torch tensors from K_0 data to train
    K_torch_features = torch.tensor(K_0_features.values, dtype=torch.float)
    K_torch_labels = torch.tensor(K_0_labels.values, dtype=torch.float)

    # Training the PyTorch model with K_0 data
    print('Training Model on K_0 Data')
    train_model(model, epochs, loss_fn, optimizer, K_torch_features, K_torch_labels)

    print('\n\nNow Finding K-K_0 Samples')

    # Renaming K_0 data to K data since we will be adding to it
    K_features, K_labels= K_0_features, K_0_labels

    # Sampling the K-K_0 samples
    #K_0, K+1
    for k in range(K_0+1, K+1):
        # This will save all the distances for this iteration
        GSy_distances = []
        
        for sample_idx in features.index:

            unlabeled_sample = features.loc[sample_idx].values
            unlabeled_prediction = model(torch.tensor(unlabeled_sample, dtype=torch.float))

            # Computing the minimum distances of current unlabelled sample's prediction and all K samples labels
            min_distance = distance_GSy(unlabeled_prediction.detach().numpy(), K_labels)
                
            # Appending the minimum distance
            GSy_distances.append(min_distance)

        # Grabbing the sample idx with the largest minimum GSy distance
        new_k_sample_idx = features.index[np.argmax(GSy_distances)]
        
        # Placing new k sample and label into the K dataframes
        new_k_sample = features.loc[new_k_sample_idx]
        new_k_label = true_dataset.loc[new_k_sample_idx, label_cols]
        K_features = K_features.append(new_k_sample)
        K_labels = K_labels.append(new_k_label)

        # Delecting the new k sample from features
        features = features.drop(new_k_sample_idx, axis=0)
        
        # Retraining PyTorch model with new value
        print('Retraining PyTorch model for new k sample, k={}'.format(k))
        K_torch_features = torch.tensor(K_features.values, dtype=torch.float)
        K_torch_labels = torch.tensor(K_labels.values, dtype=torch.float)
        train_model(model, epochs, loss_fn, optimizer, K_torch_features, K_torch_labels)

    return model, features, pd.concat([K_features, K_labels], axis=1)

In [44]:
# K can be a subset of the original data, I am choosing 20% of original data
K=int(df.shape[0] * 0.2)
output = GSy_Algorithm(features, K, model, true_dataset=df, label_cols=['Age', 'Strength'], epochs=50)
model, unlabeled_data, labeled_data = output

Running GSx Algorithm to get K_0 data
Training Model on K_0 Data


Now Finding K-K_0 Samples
Retraining PyTorch model for new k sample, k=8
Retraining PyTorch model for new k sample, k=9
Retraining PyTorch model for new k sample, k=10
Retraining PyTorch model for new k sample, k=11
Retraining PyTorch model for new k sample, k=12
Retraining PyTorch model for new k sample, k=13
Retraining PyTorch model for new k sample, k=14
Retraining PyTorch model for new k sample, k=15
Retraining PyTorch model for new k sample, k=16
Retraining PyTorch model for new k sample, k=17
Retraining PyTorch model for new k sample, k=18
Retraining PyTorch model for new k sample, k=19
Retraining PyTorch model for new k sample, k=20
Retraining PyTorch model for new k sample, k=21
Retraining PyTorch model for new k sample, k=22
Retraining PyTorch model for new k sample, k=23
Retraining PyTorch model for new k sample, k=24
Retraining PyTorch model for new k sample, k=25
Retraining PyTorch model for new k sample, k=

In [45]:
labeled_data

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
985,-0.247524,0.288646,0.356449,0.329336,0.049439,-0.693456,-0.302840,28.0,33.80
76,1.797327,0.501907,-0.846733,-2.049585,4.351528,-1.553862,0.834658,3.0,40.20
751,2.476712,-0.856472,-0.846733,-0.401199,-1.038638,1.955927,-2.002850,7.0,52.61
827,2.304473,-0.856472,-0.846733,-1.665586,-1.038638,-0.989261,1.526885,7.0,50.51
224,-1.082880,-0.368522,1.712762,-2.798851,-0.084478,1.103237,0.081315,3.0,7.75
...,...,...,...,...,...,...,...,...,...
926,-1.140293,1.623844,1.715887,0.956847,0.635327,-1.966703,-1.167189,28.0,30.65
857,-1.427357,1.542712,1.669010,-0.120224,-0.201656,-1.362232,-0.468725,28.0,33.30
924,-1.389082,1.415219,0.684589,0.816359,-0.034259,-1.619454,0.117485,28.0,26.97
904,-1.207274,1.264546,-0.846733,0.535385,0.467930,-1.233621,1.177653,28.0,23.79


In [46]:
unlabeled_data_tensor = torch.tensor(unlabeled_data.values, dtype=torch.float)
labeled_data_tensor = torch.tensor(labeled_data.values, dtype=torch.float)
predictions = model(unlabeled_data_tensor)
real_labels = torch.tensor(df.loc[unlabeled_data.index, ['Age', 'Strength']].values, dtype=torch.float)
loss = loss_fn(predictions, real_labels)
print('Loss:', loss)

Loss: tensor(3501.6204, grad_fn=<MseLossBackward0>)


In [38]:
real_labels[:5]

tensor([[ 28.0000,  79.9900],
        [ 28.0000,  61.8900],
        [360.0000,  44.3000],
        [ 90.0000,  38.0700],
        [ 28.0000,  28.0200]])

In [39]:
predictions[:5]

tensor([[17.2830, 13.1119],
        [15.7924, 13.1427],
        [20.2455,  0.8679],
        [20.2455,  0.8679],
        [20.2455,  0.8679]], grad_fn=<SliceBackward0>)

## iGS Algorithm

This is a combination of both GSx and GSy where it finds the best samples to label based on both feature and output space. It is almost identical to GSy where it uses GSx to get the initial K_0 set, but the minimum distance calculated is not a distance but the product of the two distances of GSx and GSy.

In [56]:
# This is the function that computes distance for iGS algorithm (Equation 2 in Section 2.3)
def distance_iGS(unlabeled_sample, unlabeled_prediction, K_features, K_labels):
    """
    This function combines both distances of GSx and GSy

    params:
        unlabeled_sample (np.array) - The unlabelled sample that is used to find the closest sampled K sample
        unlabeled_prediction (np.array) - The unlabelled prediction that is used to find the closest sampled K label
        K_features (pd.DataFrame) - A DataFrame of all K samples
        K_labels (pd.Series or pd.DataFrame) - A Series or DataFrame of all labels collected for the K samples

    returns:
        The minimum distance of all the distances between the unlabelled sample's prediction and all K samples' predictions
    """
    distances = []

    # If there are multiple targets, will use euchlidean distance
    for i in range(K_features.shape[0]):

        # Getting k labels and samples at ith index
        k_label = K_labels.values[i]
        k_sample = K_features.values[i]

        # GSx distance
        GSx_distance = np.linalg.norm(k_sample - unlabeled_sample)

        # GSy distance
        # If there are multiple targets, will use euchlidean distance
        if K_labels.values.ndim > 1:
            GSy_distance = np.linalg.norm(unlabeled_prediction - k_label)
        else:
            GSy_distance = np.abs(unlabeled_prediction - k_label)

        # Product of both
        distances.append(GSx_distance*GSy_distance)

    return min(distances)

# This will use iGS method from paper to get a model able to make predictions from 
def iGS_Algorithm(features, K, model, true_dataset=None, label_cols=None, K_0=None, epochs=None):
    """
    params:
        features (pd.DataFrame) - This is the dataset that is unlabelled, which will provide samples to hand label
                                before constructing a model to then use predictions to label
        K (int) - This is the total amount of samples that would be hand labelled before constructing a model
        model (PyTorch Model) - This is the model that will be trained overtime
        true_dataset (pd.DataFrame) - This is the real dataset to sample labels from since this is for simulation
        label_cols (list) - A list of all label column names in true_dataset to label the K samples
        K_0 (int) - The number of initial k samples that are sampled from the GSx algorithm to then train the model.
                    According to the authors, it is set to the number of features in the dataset.
        epochs (int) - The number of epochs to train the model for each update

    returns:
        This will return a PyTorch model trained on the K labelled Samples, the leftover unlabelled data to predict,
        and the Final K dataset
    """
    
    # Initializing K_0 to the number of features
    if K_0 == None:
        K_0 = features.shape[1]

    print('Running iGS Algorithm to get K_0 data')
    # Running GSx algorithm to collect K_0 samples
    K_0_features, K_0_labels = GSx_Sampling(features, K=K_0, true_dataset=true_dataset, label_cols=label_cols)

    # Removing labeled K_0 data from features
    features = features.drop(index = K_0_features.index)

    # Creating Torch tensors from K_0 data to train
    K_torch_features = torch.tensor(K_0_features.values, dtype=torch.float)
    K_torch_labels = torch.tensor(K_0_labels.values, dtype=torch.float)

    # Training the PyTorch model with K_0 data
    print('Training Model on K_0 Data')
    train_model(model, epochs, loss_fn, optimizer, K_torch_features, K_torch_labels)

    print('\n\nNow Finding K-K_0 Samples')

    # Renaming K_0 data to K data since we will be adding to it
    K_features, K_labels = K_0_features, K_0_labels

    # Sampling the K-K_0 samples
    
    for k in range(K_0+1, K+1):
        # This will save all the distances for this iteration
        iGS_distances = []
        
        for sample_idx in features.index:

            unlabeled_sample = features.loc[sample_idx].values
            unlabeled_prediction = model(torch.tensor(unlabeled_sample, dtype=torch.float)).detach().numpy()

            # Computing the minimum product of both GSx and GSy distances
            min_distance = distance_iGS(unlabeled_sample, unlabeled_prediction, K_features, K_labels)
            
            # Appending the minimum distance
            iGS_distances.append(min_distance)

        # Grabbing the sample idx with the largest minimum iGS distance
        new_k_sample_idx = features.index[np.argmax(iGS_distances)]
        
        # Placing new k sample and label into the K dataframes
        new_k_sample = features.loc[new_k_sample_idx]
        new_k_label = true_dataset.loc[new_k_sample_idx, label_cols]
        K_features = K_features.append(new_k_sample)
        K_labels = K_labels.append(new_k_label)

        # Delecting the new k sample from features
        features = features.drop(new_k_sample_idx, axis=0)
        
        # Retraining PyTorch model with new value
        print('Retraining PyTorch model for new k sample, k={}'.format(k))
        K_torch_features = torch.tensor(K_features.values, dtype=torch.float)
        K_torch_labels = torch.tensor(K_labels.values, dtype=torch.float)
        train_model(model, epochs, loss_fn, optimizer, K_torch_features, K_torch_labels)

    return model, features, pd.concat([K_features, K_labels], axis=1)

In [57]:
# K can be a subset of the original data, I am choosing 20% of original data
K=int(df.shape[0] * 0.2)
output = iGS_Algorithm(features, K, model, true_dataset=df, label_cols=['Age', 'Strength'], epochs=50)
model, unlabeled_data, labeled_data = output

Running iGS Algorithm to get K_0 data
Training Model on K_0 Data


Now Finding K-K_0 Samples
Retraining PyTorch model for new k sample, k=8
Retraining PyTorch model for new k sample, k=9
Retraining PyTorch model for new k sample, k=10
Retraining PyTorch model for new k sample, k=11
Retraining PyTorch model for new k sample, k=12
Retraining PyTorch model for new k sample, k=13
Retraining PyTorch model for new k sample, k=14
Retraining PyTorch model for new k sample, k=15
Retraining PyTorch model for new k sample, k=16
Retraining PyTorch model for new k sample, k=17
Retraining PyTorch model for new k sample, k=18
Retraining PyTorch model for new k sample, k=19
Retraining PyTorch model for new k sample, k=20
Retraining PyTorch model for new k sample, k=21
Retraining PyTorch model for new k sample, k=22
Retraining PyTorch model for new k sample, k=23
Retraining PyTorch model for new k sample, k=24
Retraining PyTorch model for new k sample, k=25
Retraining PyTorch model for new k sample, k=

#Analyzing and differentiating from different Techniques

Next Steps:

1. Create a Baseline model with entire dataset to find best hyperparameters for the neural network.
2. GSx Model
3. GSy Model
4. iGS Model
5. Inferencing from variables to see how they correlate to the increase or decrease of both targerts
5. Investigating which technique leads to higher performance
6. Plotting what algorithm leads to a faster convergence
7. Investigate how K effects both convergence and performance of all the models